# 실습 8: 합성곱과 풀링의 구현

본 실습에서 사용하는 이미지의 출처는 다음과 같음을 밝힌다.

Photo by Andriyko Podilnyk on Unsplash

이 실습에서는 기본적인 합성곱(convolution) 연산을 직접 작성하여 이미지에 적용해본다. 그런 다음 MyGrad에서 제공하는 합성곱 함수와 비교할 것이다. 또한, 최대 풀링(max-pooling) 작업을 구현하고, MyGrad와 비교할 것이다.

이번 실습에서 필요한 기본적인 라이브러리들을 먼저 import 하자.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import mygrad
import mygrad as mg
%matplotlib notebook

### Step 1. 윈도우(Window)

이미지에서 합성곱의 단일 곱셈-누산과 최대 풀링이 이루어지는 특정 크기의 영역을 윈도우(window)라고 부른다. 합성곱 연산에서는 필터가 겹쳐진 영역이 윈도우에 해당하며, $n \times m$ 풀링 연산의 경우에는 풀링 연산을 수행하기 위해 주목하고 있는 $n \times m$영역이 윈도우에 해당한다. Step 1에서는 이미지에서 윈도우에 해당하는 픽셀들만 선택할 때 사용하기 편리한 `mygrad.sliding_window_view`함수를 사용해본다. 연습해보자.

우리는 실습 1에서 다운로드한 데이터 중 cat.png 사진 파일을 사용해볼 것이다. 이 파일의 경로를 아래 코드와 같이 mpimg.imread()함수로 전달하면 이미지 데이터를 NumPy의 ndarray 형식으로 불러올 수 있다. 이렇게 불러온 이미지를 matplotlib로 그려보자.

In [ ]:
img = mpimg.imread('./pics/cat.png') # 자신의 컴퓨터에서의 경로에 맞추어 수정

fig, ax = plt.subplots()
ax.imshow(img);

그다음, 이미지의 shape을 확인하여 가로 세로 픽셀 수를 알아보자. shape은 (H(세로), W(가로), C(색깔)) 이다.

In [ ]:
img.shape

필터와의 단일 곱셈-누산을 하거나 최대 풀링 연산을 할 때 작업을 수행할 윈도우 영역의 픽셀들을 그때그때 골라서 진행할 수도 있지만, 미리 해당 윈도우들을 배열로 저장해둘 수도 있다. 이를 도와주는 함수가 바로 'mygrad.sliding_window_view'이다.

mygrad.sliding_window_view()는 이미지, 윈도우의 shape, 스트라이드(step)를 전달받아 윈도우들로 이루어진 numpy 배열을 반환하는 함수이다. window_shape은 (24, 24), step은 24로 하여 함수를 실행해보자. 이때, 위에서 불러온 cat.png 파일의 shape이 (427, 640, 3)이고, 427 = 24 * 17.791667, 640 = 24 * 26.666667 이므로 3개의 채널에 대해 17x26개의 윈도우가 만들어질 것임을 알 수 있다. 참고로, mygrad.sliding_window_view()를 이용할 때는 이미지의 shape를 $(H, W, C)$가 아니라 $(C, H, W)$로 바꾸어주어야 함에 주의하자.

In [ ]:
from mygrad import sliding_window_view
x  = img.transpose(2, 0, 1)  # (H, W, C) --> (C, H, W)
windowed_img = sliding_window_view(x, window_shape=(24, 24), step=24)

함수 실행 결과인 windowed_img의 shape을 확인해보자. shape은 (윈도우의 세로 개수, 윈도우의 가로 개수, 색깔 채널 수, 윈도우의 세로 픽셀 수, 윈도우의 가로 픽셀 수)이다.

In [ ]:
windowed_img.shape

우리는 위 코드를 통해 shape-(24, 24)인 윈도우 17x26개를 17x26 공간에 위치시켰음을 확인하였다. 각각의 윈도우를 17x26개 공간에 시각화해보자.

In [ ]:
fig,ax = plt.subplots(nrows=windowed_img.shape[0], ncols=windowed_img.shape[1])
for i,j in np.ndindex(windowed_img.shape[:2]):
    ax[i,j].imshow(windowed_img[i,j].transpose(1, 2, 0))  # (C, Hw, Ww) -> (Hw, Ww, C)
    ax[i,j].axis('off')

이 밖에도 window_shape을 (48, 48), step을 12로 하는 등, 다양하게 함수를 실행해보자.

### Step 2. 합성곱 함수 직접 만들기

#### 합성곱 수행 결과의 shape

합성곱(Convolution)을 수행하기 위해 가장 먼저 해야 할 일은 입력된 이미지의 shape과 필터의 shape을 고려할 때 출력되는 shape이 어떻게 될지 파악하는 것이다. 이론 부분에서 우리는, 합성곱 연산을 수행할 때 필터를 주어진 스트라이드만큼씩 미끄러지게 하면서 필터가 위치한 이미지 영역(윈도우)과 필터의 단일 곱셈-누산을 진행할 것임을 배웠다.

입력 이미지의 shape을 ${shape}_{in}=(H_{in}, W_{in})$, 필터의 shape을 ${shape}_{filter}=(H_{filter}, W_{filter})$, 스트라이드를 S라고 했을 때, 출력 이미지의 shape인 ${shape}_{out}=(H_{out}, W_{out})$은 다음의 공식으로부터 구할 수 있다. (여기서는 패딩을 고려하지 않도록 하자.)

\begin{equation}
H_{out} = \frac{H_{in}-H_{filter}}{S} + 1,
W_{out} = \frac{W_{in}-W_{filter}}{S} + 1
\end{equation}

$stride=(S, S)$라는 튜플을 정의하면, 위의 두 식을 하나로 합쳐서 표현할 수 있다. 이때 x축과 y축의 스트라이드가 $stride=(S_y, S_x)$로 서로 다르다고 생각하더라도 공식이 동일하게 성립한다.

\begin{equation}
shape_\text{out} = \frac{shape_\text{in} - shape_\text{filter}}{stride} + 1
\end{equation}

여기서 출력 shape이 유효한 치수인지 확인하는 것이 중요하다. 즉, 분수 혹은 음수가 되면 안 된다. 이제 출력 shape을 계산할 함수를 정의해보자. 출력 shape을 계산하는 함수는 아래에서 합성곱 함수를 구현할 때 사용될 것이다.

In [ ]:
def get_outshape(x_shape, w_shape, stride):
    '''
    입력되는 텐서의 shape과 합성곱 필터의 shape, 스트라이드를 바탕으로
    출력되는 텐서의 shape을 계산하는 공식을 그대로 구현
    
    매개변수 (Parameters)
    ----------
    x_shape : Tuple[int, int]
        입력 텐서(x) 의 shape
        
    w_shape : Tuple[int, int]
        합성곱 필터(w) 의 shape
        
    stride : Tuple[int, int]
        스트라이드
        
    반환 값 (Returns)
    -------
    numpy.ndarray[int], shape=(2,)
        shape이 x_shape인 입력 텐서에 대해 shape이 w_shape인 합성곱 필터로
        합성곱을 진행한 결과인 출력 텐서의 shape
    '''
    x_shape = np.array(x_shape)
    w_shape = np.array(w_shape)
    stride = np.array(stride)
    
    # out_shape 구하기
    
    # 여기에 코드 작성
    
    
    # 유효한 치수를 구했는지 확인하고, 유효하지 않으면 에러 발생
    if not all(i.is_integer() and i > 0 for i in out_shape):
        msg = "Stride and kernel dimensions are incompatible: \n"
        msg += "Input dimensions: {}\n".format(tuple(x_shape))
        msg += "Stride dimensions: {}\n".format(tuple(stride))
        msg += "Kernel dimensions: {}\n".format(tuple(w_shape))
        raise ValueError(msg)

    return out_shape.astype(np.int32)

원하는 기능이 잘 작동하는지 확인해보자. 아래 코드들을 실행시켜 정답이 출력되는지 살펴보자.

In [ ]:
# 입력 (10,10), 필터 (3,3), 스트라이드 (1,1)
# 출력 (8,8)
get_outshape((10, 10), (3, 3), (1, 1))

In [ ]:
# 입력 (11,11), 필터 (3,3), 스트라이드 (2,2)
# 출력 (5,5)
get_outshape((11, 11), (3, 3), (2, 2))

In [ ]:
# 입력 (10,10), 필터 (5,5), 스트라이드 (1,1)
# 출력 (6,6)
get_outshape((10, 10), (5, 5), (1, 1))

In [ ]:
# 입력 (5,5), 필터 (10,10), 스트라이드 (1,1)
# 출력 ValueError
get_outshape((5, 5), (10, 10), (1, 1))

#### 합성곱 함수 작성하기

이제 합성곱을 수행하는 함수 convolve()를 정의해보자.

이 함수는 입력값으로 합성곱을 수행할 이미지와 합성곱 필터, 스트라이드를 필요로 한다. 입력 이미지의 shape은 $(C, H, W)$, 합성곱 필터의 shape은 $(C, H_f, W_f)$, 스트라이드의 shape은 $(S_y, S_x)$ 라고 할 때, 출력되는 배열의 shape은 위에서 정의한 함수를 통해 알 수 있다. 위의 함수를 통해 구한 shape을 바탕으로 비어있는 배열을 만들고난 후, 차례대로 배열을 채워나가도록 구현해보자.

HINT > 스트라이드가 $S_x = S_y$인 경우, Step 1에서 살펴본 sliding_window_view() 함수로 윈도우 배열을 윈도우를 하나씩 꺼내면 필터와의 단일 곱셈-누산을 수행할 수 있다. 일반적으로 스트라이드는 x축과 y축에 대해 동일하게 설정하기 때문에, 아래 코드를 사용해도 충분하다.

``` python
if stride[0] == stride[1]:
        windowed_img = sliding_window_view(input_image, window_shape=conv_shape, step=stride[0])
        for ind in np.ndindex(out.shape):
            out[ind] = np.sum(conv_filter * windowed_img[ind])
```
    
하지만 NumPy 배열을 다루는 데 더 익숙해지고 싶다면 $S_x \neq S_y$인 경우까지 고려하여 코드를 새롭게 작성해보자. 입력 이미지 중에서 윈도우에 해당하는 픽셀들을 그때그때 선택해주어야 한다. 이때, 축이 세 개인 input_image 배열을 슬라이싱 하기 위해 slice() 함수를 사용할 수 있다. 각 축에서 슬라이싱할 범위를 slice() 함수로 표현하여 원소가 세개인 튜플을 구성하면 된다.

In [ ]:
def convolve(input_image, conv_filter, stride=(1, 1)):
    '''
    `conv_filter`를 `stride`만큼 이동하며
    `input_image`에 대해 합성곱 연산을 진행
    
    매개변수 (Parameters)
    ----------
    input_image : numpy.ndarray, shape=(C, H, W)
        합성곱 연산을 수행할 입력 이미지
        
    conv_filter : numpy.ndarray, shape=(C, Hf, Wf)
        이미지 위에서 움직일 합성곱 필터
    
    stride : Tuple[int, int], 별다른 입력 없으면 default=(1, 1)
        합성곱 필터를 움직일 보폭(스트라이드)
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape=(H', W')
        합성곱 결과
        (H', W')는 함수 `get_outshape`를 통해 얻은 결과
    '''
    # 함수 get_outshape() 이용하여 out_shape 구하기
    
    # 여기에 코드 작성
    
    
    # 출력값을 채워넣을 비어있는 배열 out 생성
    out = np.empty(out_shape, np.float32)
    
    
    # 배열 out에 컨볼루션 연산한 값 채워넣기
    
    # 여기에 코드 작성
    
    
    return out

shape-(1,9,9)의 배열이 "image"로 주어졌다고 하자. 그리고 shape-(1,3,3)의 배열이 "conv-filter"로 주어졌다고 하자. 이때 스트라이드를 변화시켜 가면서 함수 convolve()가 잘 만들어졌는지 확인해보자. 합성곱 연산 진행 결과를 손으로도 직접 계산해보고, convolve()함수를 이용한 결과와 비교해보자.

"image"와 "conv-filter"의 예시를 하나씩 들어주자면 다음과 같다.

In [ ]:
# Ex) 이미지
image = np.array([[8,6,4,5,1,2,7,0,3],[7,2,5,3,6,0,1,4,8],[0,3,1,7,4,8,2,6,5],[6,1,8,4,3,5,0,7,2],[4,5,0,2,7,1,3,8,6],[2,7,3,8,0,6,4,5,1],[3,8,6,0,2,7,5,1,4],[5,0,2,1,8,4,6,3,7],[1,4,7,6,5,3,8,2,0]])
image = image[np.newaxis, :, :]
image

\begin{pmatrix}
8&6&4&5&1&2&7&0&3\\
7&2&5&3&6&0&1&4&8\\
0&3&1&7&4&8&2&6&5\\
6&1&8&4&3&5&0&7&2\\
4&5&0&2&7&1&3&8&6\\
2&7&3&8&0&6&4&5&1\\
3&8&6&0&2&7&5&1&4\\
5&0&2&1&8&4&6&3&7\\
1&4&7&6&5&3&8&2&0
\end{pmatrix}

In [ ]:
# Ex) 필터
conv_filter = np.array([[1,2,0],[0,1,2],[2,0,1]])
conv_filter = conv_filter[np.newaxis, :, :]
conv_filter

\begin{pmatrix}
1&2&0\\
0&1&2\\
2&0&1
\end{pmatrix}

다음의 각 질문에 대답하기 위해 손으로도 계산해보고, convolve() 함수도 잘 이용해보자.

1) 스트라이드를 1 (stride=(1,1))로 하면 합성곱 진행 결과가 어떻게 나타날까?

In [ ]:
# 여기에 코드 작성

2) 스트라이드를 2 (stride=(2,2))로 하면 합성곱 진행 결과가 어떻게 나타날까?

In [ ]:
# 여기에 코드 작성

3) 스트라이드를 3 (stride=(3,3))로 하면 합성곱 진행 결과가 어떻게 나타날까?

In [ ]:
# 여기에 코드 작성

#### 이미지 집합과 필터 뱅크에 대한 합성곱 함수 작성하기

위에서 우리는 합성곱 함수를 정의하였다. 이제 이 함수를 개선하여, 이미지 집합(이미지 여러 개)에 필터 뱅크(필터 여러 개)를 한 번에 적용할 수 있는 함수를 만들어볼 것이다. 이 함수는 CNN의 합성곱 계층으로 사용할 수 있을 것이다.

아래 conv_bank()함수는 각 $N$개의 이미지 $(C, H, W)$에 대해 $K$개의 필터 $(C, H_f, W_f)$를 반복하여 단일 합성곱 계층의 전체 순전파를 수행하는 함수이다. 주석을 잘 읽고, 위의 convolve()함수를 개선하여 코드를 완성해보자. 합성곱 기능을 다시 프로그래밍 할 필요 없이, convolve()함수를 이용하여 합성곱 연산을 수행하도록 작성해보자.

HINT > for문에서 enumerate()함수를 이용하면 이미지 집합과 필터 뱅크에서 각 이미지 및 필터와 해당하는 인덱스를 한 번에 꺼내어 프로그래밍하는 데 유리할 것이다.

In [ ]:
def conv_bank(input_images, conv_filters, stride=(1, 1)):
    '''
    `conv_filter` 여러 개를 `stride`만큼 이동하며
    `input_image` 여러 개에 대해 합성곱 연산을 진행
    
    매개변수 (Parameters)
    ----------
    input_images : numpy.ndarray, shape=(N, C, H, W)
        합성곱을 수행할 입력 이미지들
        
    conv_filters : numpy.ndarray, shape=(K, C, Hf, Wf)
        이미지 위에서 움직일 합성곱 필터들
    
    stride : Tuple[int, int], 별다른 입력 없으면 default=(1, 1)
        합성곱 필터를 움직일 스트라이드
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape=(N, K, H', W')
        합성곱 결과
        (H', W')는 함수 `get_outshape`를 통해 얻은 결과
    '''
    # 함수 get_outshape() 이용하여 out_shape 구하기
    
    # 여기에 코드 작성
    
    
    # 출력값을 채워넣을 비어있는 배열 out 생성
    out = np.empty((len(input_images), len(conv_filters), *out_shape))
    
    
    # 배열 out에 컨볼루션 연산한 값 채워넣기
    
    # 여기에 코드 작성
    
    
    return out

이제 이전과 같은 방식으로 함수의 동작을 확인할 수 있다.

shape-(2,1,9,9)의 배열이 이미지 집합 "input_images"로 주어졌다고 하자. 그리고 shape-(4,1,3,3)의 배열이 필터 뱅크 "conv-filters"로 주어졌다고 하자. 이미지 집합 "input_image"와 필터 뱅크 "conv-filter"의 예시를 하나씩 들어주자면 다음과 같다.

In [ ]:
# Ex) 이미지 스택
input_images = np.array([[[8,6,4,5,1,2,7,0,3],[7,2,5,3,6,0,1,4,8],[0,3,1,7,4,8,2,6,5],[6,1,8,4,3,5,0,7,2],[4,5,0,2,7,1,3,8,6],[2,7,3,8,0,6,4,5,1],[3,8,6,0,2,7,5,1,4],[5,0,2,1,8,4,6,3,7],[1,4,7,6,5,3,8,2,0]], [[0,2,0,1,1,2,3,0,3],[3,2,1,3,2,0,1,0,0],[0,3,1,3,0,0,2,2,1],[2,1,0,0,3,1,0,3,2],[0,1,0,2,3,1,3,0,2],[2,3,3,0,0,2,0,1,1],[3,0,2,0,2,3,1,1,0],[1,0,2,1,0,0,2,3,3],[1,0,3,2,1,3,0,2,0]]])
input_images = input_images[:, np.newaxis, :, :]
input_images

\begin{pmatrix}
8&6&4&5&1&2&7&0&3\\
7&2&5&3&6&0&1&4&8\\
0&3&1&7&4&8&2&6&5\\
6&1&8&4&3&5&0&7&2\\
4&5&0&2&7&1&3&8&6\\
2&7&3&8&0&6&4&5&1\\
3&8&6&0&2&7&5&1&4\\
5&0&2&1&8&4&6&3&7\\
1&4&7&6&5&3&8&2&0
\end{pmatrix}
\
\begin{pmatrix}
0&2&0&1&1&2&3&0&3\\
3&2&1&3&2&0&1&0&0\\
0&3&1&3&0&0&2&2&1\\
2&1&0&0&3&1&0&3&2\\
0&1&0&2&3&1&3&0&2\\
2&3&3&0&0&2&0&1&1\\
3&0&2&0&2&3&1&1&0\\
1&0&2&1&0&0&2&3&3\\
1&0&3&2&1&3&0&2&0
\end{pmatrix}

In [ ]:
# Ex) 필터 뱅크
conv_filters = np.array([[[1,2,0],[0,1,2],[2,0,1]],[[-1,-1,-1],[-1,8,-1],[-1,-1,-1]],[[0,-1,0],[-1,5,-1],[0,-1,0]],[[1/16,2/16,1/16],[2/16,4/16,2/16],[1/16,2/16,1/16]]])
conv_filters = conv_filters[:, np.newaxis, :, :]
conv_filters

\begin{pmatrix}
1&2&0\\
0&1&2\\
2&0&1
\end{pmatrix}
\
\begin{pmatrix}
-1&-1&-1\\
-1&8&-1\\
-1&-1&-1
\end{pmatrix}
\
\begin{pmatrix}
0&-1&0\\
-1&5&-1\\
0&-1&0
\end{pmatrix}
\
\begin{pmatrix}
1/16&2/16&1/16\\
2/16&4/16&2/16\\
1/16&2/16&1/16
\end{pmatrix}

이때 스트라이드 3 (stride=(3,3))에 대해 함수 conv_bank()가 잘 만들어졌는지 확인해보자. 합성곱 연산 진행 결과를 손으로도 직접 계산해보고, conv_bank() 함수를 이용한 결과와 비교해보자.

In [ ]:
# 여기에 코드 작성

#### 합성곱 함수 성능 평가

이제 소요 시간을 기준으로 위에서 만든 conv_bank 함수와 MyGrad의 합성곱 함수의 성능을 비교해보자. MyGrad의 함수가 더 짧은 시간이 걸림을 확인할 수 있을 것이다.

(참고) 소요 시간을 측정하기 위해 timeit 함수를 사용한다. timeit 함수는 코드를 여러 번 반복 실행한 후 소요시간의 평균과 표준편차를 반환해준다. jupyter notebook에서는 매직 커맨드(magic commend) %%을 이용하여 셀(cell) 내의 모든 코드를 실행하는 데 소요된 시간을 측정할 수 있다.

In [ ]:
timing_images = np.random.rand(10, 3, 64, 64)
timing_filters = np.random.rand(20, 3, 3, 3)

In [ ]:
%%timeit
conv_bank(timing_images, timing_filters)

In [ ]:
from mygrad.nnet.layers import conv_nd

In [ ]:
%%timeit
conv_nd(timing_images, timing_filters, stride=(1, 1))

### Step 3. 최대 풀링 함수 직접 만들기

#### 풀링 수행 결과의 shape

풀링(pooling)을 수행하기 위해 가장 먼저 해야 할 일은 입력된 이미지의 shape과 풀링 윈도우의 shape을 고려할 때 출력되는 shape이 어떻게 될지 파악하는 것이다.

풀링을 수행할 때는 윈도우를 주어진 스트라이드만큼씩 미끄러지게 하면서 풀링을 진행한다. 이 실습에서 풀링 연산의 스트라이드는 풀링 윈도우의 가로, 세로 길이와 동일한 경우만 고려할 것이다.

따라서, 입력 이미지의 shape이 $shape_{in} = (H_{in}, W_{in})$, 풀링 윈도우의 shape이 $shape_{pool} = (H_{pool}, W_{pool})$, 스트라이드가 $stride = (H_{pool}, W_{pool})$라면, 출력 이미지의 shape인 $shape_{out} = (H_{out}, W_{out})$은 다음의 공식으로부터 구할 수 있다. (일반적으로 풀링 윈도우의 가로와 세로 길이가 동일하지만, 실습에서는 가로와 세로의 길이가 같든 다르든 동일하게 적용이 가능한 일반적인 코드를 작성함으로써 파이썬 프로그래밍에 더 익숙해져 보자.)

\begin{equation}
H_{out} = \frac{H_{in} - H_{pool}}{H_{pool}} + 1,
W_{out} = \frac{W_{in} - W_{pool}}{W_{pool}} + 1
\end{equation}

그리고 위의 두 식을 하나로 합쳐서 표현하면 다음과 같은 공식을 얻을 수 있다.

\begin{equation}
shape_\text{out} = \frac{shape_\text{in} - shape_\text{pool}}{shape_\text{pool}} + 1 = \frac{shape_\text{in}}{shape_\text{pool}}
\end{equation}

#### 풀링 함수 작성하기

이제 최대 풀링(max-pooling)을 수행하는 함수 max_pool()를 정의해보자. 최대 풀링은 이미지의 크기를 줄이는 다운 샘플링에서 매우 유용하게 사용된다.

이 함수는 입력값으로 최대 풀링을 수행할 단일 채널 이미지와 풀링 윈도우의 shape을 필요로 한다. 입력 이미지의 shape은 $(H,W)$, 풀의 shape은 $(H_p,W_p)$ 라고 할 때, 출력되는 배열의 shape은 위에서 살펴본대로 "shape_in // shape_pool" 이다.

convolve() 함수와 마찬가지로 비어있는 배열을 먼저 만들고난 후, 차례대로 배열을 채워나가도록 max_pool()을 구현해보자.

HINT > 배열을 채울 때, convolve()함수와 비슷한 방식으로 프로그래밍하면 된다. slice()함수를 이용하여 배열을 슬라이싱하면 윈도우를 만들기 유리하다. 만약, 풀링 윈도우의 가로와 세로 길이가 같은 경우만 고려하고 싶다면, Step 1에서 살펴본 sliding_window_view() 함수를 사용할 수도 있다.

In [ ]:
def max_pool(input_image, pool_shape=(2, 2)):
    '''
    이미지의 단일 채널에 대해 최대 풀링을 수행
    
    매개변수 (Parameters)
    ----------
    input_image : numpy.ndarray, shape=(H, W)
        최대 풀링을 수행할 채널
        
    pool_shape : Tuple[int, int], optional (default=(2, 2))
        풀링 윈도우의 shape. (Hp, Wp)
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape=(H', W')
        최대 풀링 결과
        H' = H / Hp, W'= W / Wp
    '''
    # out_shape 구하기
    
    # 여기에 코드 작성
    
    
    # 출력값을 채워넣을 비어있는 배열 out 생성
    out = np.empty(out_shape)
    
    
    # 배열 out에 최대풀링한 값 채워넣기
    
    # 여기에 코드 작성
    
    
    return out

함수가 제대로 작동하는지 아래 코드를 이용하여 확인해보자.

In [ ]:
a = np.random.rand(4, 4)
print(a, '\n')
print(max_pool(a))

#### 이미지 집합에 대한 풀링 함수 작성하기

위에서 우리는 최대 풀링 함수를 정의하였다. 이제 이 함수를 개선하여, 이미지 집합(이미지 여러 개)의 각 이미지의 채널을 순환하며 풀링할 수 있는 함수를 만들어볼 것이다. 이 함수는 CNN의 풀링 계층으로 사용할 수 있을 것이다.

아래 max_pool_stack()함수는 각 $N$개의 이미지 $(C, H, W)$에 대해 풀링 shape을 $(H_p ,W_p)$으로 하여 $N$개 이미지 $C$개 채널에 대해 단일 최대풀링 레이어의 전체 순전파를 수행하는 함수이다. 주석을 잘 읽고, 위의 max_pool()함수를 개선하여 코드를 완성해보자.

HINT > 배열을 채울 때, conv_bank()함수와 비슷한 방식으로 프로그래밍하면 된다. for문에서 enumerate() 함수를 사용하면 유리하다.

In [ ]:
def max_pool_stack(input_images, pool_shape=(2, 2)):
    '''
    이미지 스택의 각 이미지들에 대해 최대풀링을 수행
    
    매개변수 (Parameters)
    ----------
    input_images : numpy.ndarray, shape=(N, C, H, W)
        최대풀링을 수행할 채널
        
    pool_shape : Tuple[int, int], optional (default=(2, 2))
        최대풀의 shape. (Hp, Wp)
        
    반환 값 (Returns)
    -------
    numpy.ndarray, shape=(N, C, H', W')
        최대풀링 결과
        H' = H / Hp, W'= W / Wp
    '''
    # out_shape 구하기
    
    # 여기에 코드 작성
    
    
    # 출력값을 채워넣을 비어있는 배열 out 생성
    out = np.empty(out_shape)
    
    
    # 배열 out에 최대풀링한 값 채워넣기
    
    # 여기에 코드 작성
    
    
    return out

함수가 제대로 작동하는지 아래 코드를 이용하여 확인해보자.

In [ ]:
a = np.random.rand(2, 2, 4, 4)
print(a, '\n')
print(max_pool_stack(a, (2, 2)))

#### 풀링 함수 성능 평가

이제 소요 시간을 기준으로 위에서 만든 max_pool_stack 함수와 MyGrad의 최대 풀링 함수의 성능을 비교해보자.

In [ ]:
a = np.random.rand(20, 10, 64, 64)

In [ ]:
%%timeit
max_pool_stack(a, (2, 2))

In [ ]:
from mygrad.nnet.layers import max_pool

In [ ]:
%%timeit
max_pool(a, (2, 2), (2, 2))

### Step 4. 합성곱 연산을 이용한 Image Processing

합성곱 연산 수행을 통해 사용되는 필터 중, 가장자리를 검출하는 등, 이미지 프로세싱의 측면에서 이미지를 특정 목적에 따라 가공해주는 필터들이 알려져 있다. 우리는 5절 이론에서 CNN을 학습시키면 이미지의 특징을 추출해주는 필터를 얻을 수 있게 된다는 것을 배웠고, 이러한 필터들 중 일부는 적절히 학습시키면 잘 알려진 필터에 가까워지기도 한다는 사실이 발견되었다.

몇가지 유용하고 간단한 3x3 필터를 살펴보자.

1. 이미지의 가장자리(edge)를 검출하는 필터
```python
edge_detect_kernel = np.array([[-1, -1, -1],
                 [-1,  8, -1],
                 [-1, -1, -1]])
```

2. 이미지를 선명하게(sharpen) 만드는 필터
```python
sharpening_kernel = np.array([[ 0, -1,  0],
                  [-1,  5, -1],
                  [ 0, -1,  0]])
```

3. 이미지를 흐리게(blur) 만드는 필터
```python
gauss_blur = np.array([[1, 2, 1],
                       [2, 4, 2],
                       [1, 2, 1]])
gauss_blur = gauss_blur / 16
```

이러한 필터들을 이용하여 직접 이미지 프로세싱을 진행해보기 위해 합성곱을 수행하는 함수를 사용해볼 것이다. 위에서 우리가 만들었던 함수와 MyGrad의 conv_nd 함수 모두 사용할 수 있다.

일단 cat.png 원본 이미지를 불러와서 그려보자.

In [ ]:
# matplotlib를 이용하여 이미지 시각화하기
img = mpimg.imread('./pics/cat.png') # 자신의 컴퓨터에서의 경로에 맞추어 수정

fig, ax = plt.subplots()
ax.imshow(img);

이 이미지 배열의 shape은 무엇이고, 색깔 채널이 포함된 축은 어떤 축인지 한 줄의 코드로 재확인하자.

In [ ]:
# 여기에 코드 작성

아래 함수는 합성곱을 이용하여 주어진 필터에 대해 이미지를 프로세싱할 수 있는 함수이다. 이 함수에서 주의 깊게 살펴봐야 하는 부분은 transpose(), reshape(), squeeze() 등 numpy.ndarray의 shape을 변화시키고 축을 제거하는 작업들이다.

채널이 3개인 이미지에 대한 합성곱 연산은 채널이 3개인 필터와의 단일 곱셈-누산을 통해 이미지 채널 1개를 생성하는 작업이다. 그러나 이미지 프로세싱에서는 이미지의 채널 3개에 대해 필터 1개를 각각 적용한 후 가공된 이미지 채널 3개를 얻는 것을 목표로 한다. 따라서, 이 함수에서는 입력된 컬러 이미지 배열을, 채널이 3개인 이미지 1개가 아니라 채널이 1개인 이미지 3개로 변환하여 conv_nd() 함수에 넣어준다. 그리고 conv_nd()의 결과값으로 얻어진 채널이 1개인 이미지 3개를 다시 채널이 3개인 이미지 1개로 변환해준 후 가공된 컬러 이미지를 반환한다.

참고로, 아래에서 새롭게 등장한 squeeze()함수는 차원이 1인 축을 제거해주는 함수로, 이미지 배열을 컬러 이미지로 되돌려 주기 위한 과정에서 사용하였다. PyTorch 라이브러리에서도 같은 이름의 함수가 존재한다. 코드를 이해해보자.

In [ ]:
def image_process_conv(img, kernel):
    """
    MyGrad의 conv_nd 함수를 사용하여 이미지 프로세싱을 진행
    색깔 채널의 축소 없이 단일 2D 이미지에서 단일 2D 필터로 합성곱을 진행
    ** matplotlib.imshow 함수 사용하기 위한 이미지 축의 순서: (H, W, C)

    매개변수 (Parameters)
    ----------
    img : numpy.ndarray, shape=(H, W, C)
    kernel : numpy.ndarray, shape=(Hf, Wf)

    반환 값 (Returns)
    -------
    convolved_img : numpy.ndarray, shape=(H', W', C)
    """
    # (H, W, C) --> (C, 1, H, W)
    x = img.transpose(2,0,1)[:, np.newaxis, :, :]
    
    # (Hf, Wf) --> (1, 1, Hf, Wf)
    kernel = kernel.reshape(1, 1, *kernel.shape)
    
    # conv: (C, 1, H, W) with (1, 1, Hf, Wf) --> (C, 1, H', W')
    # squeeze : (C, 1, H', W') --> (C, H', W')
    # transpose: (C, H', W') --> (H', W', C)
    return conv_nd(x, kernel, stride=(1, 1)).data.squeeze().transpose(1, 2, 0)

방금 작성한 image_process_conv()함수를 이용하여 앞서 살펴보았던 세 종류의 필터를 이용한 이미지 프로세싱을 진행해보자.

In [ ]:
# 가장자리 검출 커널

# 여기에 코드 작성


fig, ax = plt.subplots()
ax.imshow(image_process_conv(img, edge_detect_kernel));

In [ ]:
# 선명하게 만드는 커널

# 여기에 코드 작성


fig, ax = plt.subplots()
ax.imshow(image_process_conv(img, sharpening_kernel));

In [ ]:
# 흐리게 만드는 커널

# 여기에 코드 작성


fig, ax = plt.subplots()
ax.imshow(image_process_conv(img, gauss_blur));

### 배운 내용 되돌아보기

이번 실습에서는 합성곱과 풀링의 원리를 이해하기 위해 합성곱 함수와 최대 풀링 함수를 직접 구현해보고, MyGrad에서 제공하는 함수와 비교해보았다. 또한, 이미지 프로세싱에 필터와 합성곱 연산이 사용됨을 추가적으로 배우고, 가공한 결과를 시각적으로 확인해보았다. 배운 내용을 순서대로 되돌아보자.

- 합성곱 연산과 풀링 연산이 이루어질 영역(윈도우)에 해당하는 픽셀들을 지정해주는 작업을 위해 MyGrad의 함수를 사용해보았다. 또한 그 결과로 얻어진 윈도우로 이루어진 배열을 시각화하여 살펴보기도 하였다. 이 함수는 합성곱 연산 및 최대 풀링 연산을 진행할 때 윈도우에 해당하는 픽셀들을 바로 꺼내어 사용할 수 있게 해준다는 점에서 도움이 되었다. 

- (선택적으로) 그러나 윈도우의 세로, 가로와 스트라이드의 y축, x축 값을 서로 다르게 하여 이미지의 윈도우 영역에 바로 접근할 수 있는 코드를 직접 작성해보기도 했다.

- 합성곱 수행 결과의 shape 공식을 생각해보고, 해당 shape을 갖는 빈 배열을 생성하였다. 이후, 빈 배열에 단일 곱셉-누산 결과를 하나씩 저장하여 최종적으로 합성곱 수행 결과를 반환할 수 있는 함수를 작성하였다.

- 작성한 합성곱 함수를 응용하여 이미지 집합과 필터 뱅크에 대한 합성곱 함수를 작성하였으며, 직접 작성한 합성곱 함수와 MyGrad의 합성곱 함수의 성능을 비교해보았다.

- 풀링 수행 결과의 shape 공식을 생각해보고, 해당 shape을 갖는 빈 배열을 생성하였다. 이후, 빈 배열에 최대 풀링 결과를 하나씩 저장하여 최종적으로 최대 풀링 수행 결과를 반환할 수 있는 함수를 작성하였다.

- 작성한 최대 풀링 함수를 응용하여 이미지 집합에 대한 최대 풀링 함수를 작성하였으며, 직접 작성한 최대 풀링 함수와 MyGrad의 최대 풀링 함수의 성능을 비교해보았다.

- 어떤 특징을 추출하는지 그 효과가 잘 알려진 세 종류의 3x3 필터에 대해 배웠다. 그리고 세 필터를 이용하여 합성곱 함수를 통한 이미지 프로세싱을 직접 진행해보았다.